In [3]:
!pip install --upgrade yfinance yahooquery transformers torch spacy faker --no-cache-dir
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 238.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 224.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 280.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 292.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 278.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 583.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 215.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 201.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 205.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━

In [4]:
import random
import yfinance as yf
import spacy
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from yahooquery import Screener
from faker import Faker


# Load SpaCy for named entity recognition
nlp = spacy.load("en_core_web_lg")

fake = Faker()

# Load Llama 3 Omni FinAI model and tokenizer
model_name = "ichanchiu/Llama-3.1-Omni-FinAI-8B" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Pipeline for text generation
fin_ai_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


In [15]:
# FUnction to generate random email attributes

def generate_random_email():
    sender_name = fake.name()
    receiver_name = fake.name()
    receiver_first_name = receiver_name.split()[0]
    sender_email = f"{sender_name.split()[0].lower()}.{sender_name.split()[1].lower()}@{random.choice(['wellsfargo.com'])}"
    receiver_email = f"{receiver_name.split()[0].lower()}.{receiver_name.split()[1].lower()}@{random.choice(['wellsfargo.com', 'bloomberg.net', 'reedsmith.com', 'bondxnapi.com', 'franklinetempleton.com', 'walkerdunlop.com', 'pimco.com'])}"
    return sender_email, receiver_email, sender_name, receiver_first_name

# Generate subject
def generate_subject(name):
    subject = "Discussion on the stock - " + name
    return subject

# Function to fetch stock summary from Yahoo Finance
def fetch_yahoo_finance_summary(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    summary = info.get('longBusinessSummary', '')
    name = info.get('longName','')
    if not summary:
        return "No summary available for this stock."
    income_statement = stock.financials
    balance_sheet = stock.balance_sheet
    cash_flow = stock.cashflow
    return summary, income_statement, balance_sheet, cash_flow, name

# Function to generate random financial data
def generate_random_financial_data():

    s = Screener()
    data = s.get_screeners("most_actives", count=100)
    tickers = [stock["symbol"] for stock in data["most_actives"]["quotes"]]
    #prices = [150.25, 302.15, 2800.60, 3345.00, 895.50, 365.20]
    sedol = [f"{random.randint(1000000, 9999999)}" for _ in range(6)]
    cusip = [f"{random.randint(10000000, 99999999)}" for _ in range(6)]
    isin = [f"US{random.randint(1000000000, 9999999999)}" for _ in range(6)]

    ticker = random.choice(tickers)
    price = yf.Ticker(ticker).info["currentPrice"]
    trade_sedol = random.choice(sedol)
    trade_cusip = random.choice(cusip)
    trade_isin = random.choice(isin)

    return ticker, price, trade_sedol, trade_cusip, trade_isin

# Function to generate a dynamic question based on the summary and extracted entities
def generate_dynamic_question(summary, name):
    
    doc = nlp(summary)
    
    entities = ', '.join(list(set([ent.text for ent in doc.ents])))
    
    ticker, price, trade_sedol, trade_cusip, trade_isin = generate_random_financial_data()

    # Generate a question based on the entities
    question = f"""Using these list of entities of the {name} company, entities - {entities} and its trade parameters
    ticker - {ticker}, price - {price}, sedol - {trade_sedol}, cusip - {trade_cusip}, isin - {trade_isin}
    Assume you are conversing with someone over email analyzing this company's stock and its trade parameters."""

    return question



In [17]:
# Generate random financial data
ticker, price, trade_sedol, trade_cusip, trade_isin = generate_random_financial_data()

# Fetch the Yahoo Finance summary for the input ticker
summary, income_statement, balance_sheet, cash_flow, name = fetch_yahoo_finance_summary(ticker)

# Generate a dynamic question based on the summary and name
question = generate_dynamic_question(summary, name)

# Generate a random sender and receiver email addresses
sender_email, receiver_email, sender_name, receiver_first_name = generate_random_email()

# Generate the email subject
subject = generate_subject(name)

# Generate the answer using Llama 3 model
answer = fin_ai_pipeline(question, max_new_tokens=256, device_map="auto", top_k=10, temperature=0.7, return_full_text=False, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

# Append the random trade details to the resultant answer
generated_answer = answer[0]['generated_text']
generated_answer += f"\n\nTrade Details:\nTicker: {ticker}\nPrice: ${price}\n" \
                    f"SEDOL: {trade_sedol}\nCUSIP: {trade_cusip}\nISIN: {trade_isin}"

# Final response
#print(f"Q: What are the key financial updates on {name}")
#print(f"A: {generated_answer}")

email_body = generated_answer

# Format the email
email = f"From: {sender_email}\nTo: {receiver_email}\nSubject: {subject}\n\nHi, {receiver_first_name}.\n{email_body}\n\nBest regards,\n{sender_name}"

# Output the generated email
print(email)

ValueError: The following `model_kwargs` are not used by the model: ['device_map'] (note: typos in the generate arguments will also show up in this list)